# 前処理

In [2]:
import re
import pandas as pd
import numpy as np

## データの読み込み

## Designationカラム

In [57]:
def convert_fullwidth_to_halfwidth_and_extract_invalid(
    df: pd.DataFrame,
    column_name: str,
) -> pd.DataFrame:
    """
    指定されたカラムに対して次の処理を行う:
    1. 全角英字を半角英字に変換
    2. それでもなおAからz以外の文字が含まれるユニークな値をリストとして返す
    
    Args:
        df (pd.DataFrame): 対象のデータフレーム
        column_name (str): 操作を行うカラムの名前
    
    Returns:
        pd.DataFrame: 修正後のデータフレーム
        list: 条件に合わないレコードのユニークな値
    """
    # 全角英字を半角英字に変換
    def convert_fullwidth_to_halfwidth(text: str) -> str:
        """
        全角英字を半角英字に変換するヘルパー関数

        Args:
            text (str): 入力文字列

        Returns:
            str: 半角に変換された文字列
        """
        return "".join(
            chr(ord(char) - 65248) if "Ａ" <= char <= "Ｚ" or "ａ" <= char <= "ｚ" else char
            for char in text
        )

    # 対象カラムの全角英字を半角英字に変換
    df[column_name] = df[column_name].apply(
        lambda x: convert_fullwidth_to_halfwidth(x) if pd.notna(x) else x
    )

    # 半角スペースをアンダースコアに置換
    df[column_name] = df[column_name].str.replace(" ", "_", regex=False)
    
    replace_dict = {
        "𝙧": "r",
        "α": "a",
        "Տ": "S",
        "ѵ": "v",
        "×": "x",
        "е": "e",
        "Α": "A",
        "А": "A",
        "Μ": "M",
        "Е": "E",
        "Ѕ": "S",
    }
    df = df.replace(
        {column_name: replace_dict},
        regex=True
    )
    
    # Aからz以外の文字を含む行のindexを取得
    invalid_indices = df[~df[column_name].str.match(r"^[A-Za-z_]+$", na=False)].index
    
    if len(invalid_indices) == 0:
        # ここまでで前処理が完了していれば、カラムの値を小文字に変換して返す
        df[column_name] = df[column_name].apply(
            lambda x: x.lower() if pd.notna(x) else x
        )
        return df
    else:
        # 条件に合わないレコードのユニークな値をリストとして取得
        print("there are invalid values in the column: {}".format(column_name))
        unique_invalid_values = df.loc[invalid_indices, column_name].unique().tolist()
        return df, unique_invalid_values

## MonthlyIncomeカラム

In [70]:
def extract_and_convert_to_numeric(
    df: pd.DataFrame, 
    column_name: str, 
    new_column_name: str
) -> pd.DataFrame:
    """
    指定されたカラムから数字と「万」を抽出し、1万倍して新しいカラムに保存する。
    正規表現にマッチしない場合、そのインデックスと値を記録する。

    Args:
        df (pd.DataFrame): 入力データフレーム
        column_name (str): 元のカラム名
        new_column_name (str): 結果を保存する新しいカラム名

    Returns:
        pd.DataFrame: 処理結果が保存された新しいカラムが追加されたデータフレーム
        list: 正規表現にマッチしなかったユニークな値のリスト
    """
    unmatched_values = []
    
    def convert_to_number(text: str, index: int) -> int:
        # 正規表現で「万」と数字を含む部分を抽出
        match = re.search(r"(\d+(\.\d+)?)(万)?", text)
        if not match:
            unmatched_values.append((index, text))
            return None
        number_str, _, unit = match.groups()
        number = float(number_str)
        if unit == "万":
            number *= 10000
        return int(number)

    # 各レコードに対して処理を行い、新しいカラムに保存
    df[new_column_name] = [
        convert_to_number(str(value), idx) 
        for idx, value in enumerate(df[column_name])
    ]
    df[new_column_name] = df[new_column_name].astype(np.float32)
    
    if len(unmatched_values) == 0:
        return df
    else:
        # 正規表現にマッチしなかったユニークな値をリストにして返す
        print("there are unmatched values in the column: {}".format(column_name))
        unique_unmatched_values = list({value for _, value in unmatched_values})
    
        return df, unique_unmatched_values


## Customer_infoカラム

In [48]:
def customer_info_preprocess(
    df: pd.DataFrame,  # 入力のデータフレーム
    column_name: str,  # 処理対象のカラム名
) -> pd.DataFrame:
    """
    各レコードに対して、指定されたカラムの文字列を処理し、
    各単語を条件に基づいて新しいカラムに分類する。
    
    Args:
        df (pd.DataFrame): 処理対象のデータフレーム
        column_name (str): 対象カラム名
        
    Returns:
        pd.DataFrame: 処理結果を含むデータフレーム
    """
    # 各レコードを処理
    for index, row in df.iterrows():
        # 句読点やコロン、改行などを半角スペースに変換
        cleaned_text = re.sub(r"[、。・：；,.;:?!/／\n]", " ", str(row[column_name]))
        
        # 単語に分割
        words = cleaned_text.split()

        # 各単語に対して処理を実施
        marriage_history = " ".join([word for word in words if "婚" in word or "独" in word])
        car = " ".join([word for word in words if "車" in word])
        children = " ".join([word for word in words if "婚" not in word and "独" not in word and "車" not in word])

        # 各レコードに新しいカラムを追加
        df.at[index, "marriage_history"] = marriage_history
        df.at[index, "car"] = car
        df.at[index, "children"] = children
    
    # 各カラムの表記揺れを修正
    def dict_replace_function(
        text: str,
        replace_dict: dict
    ) -> str:
        if text in replace_dict:
            return str(replace_dict[text])
        else:
            raise ValueError(f"'{text}' is not found in the replacement dictionary.")

    # car, childrenカラムの各レコードに対して置き換え処理を実施
    # car辞書の作成
    car_replace_dict = {
    "車未所持": 0,
    "自動車未所有": 0,
    "車保有なし": 0,
    "乗用車なし": 0,
    "自家用車なし": 0,
    "車なし": 0,
    "車あり": 1,
    "車所持": 1,
    "自家用車あり": 1,
    "車保有": 1,
    "乗用車所持": 1,
    "自動車所有": 1,
    }
    # children辞書の作成
    children_replace_dict = {
        "子供なし": 0,
        "子供無し": 0,
        "無子": 0,
        "子供ゼロ": 0,
        "非育児家庭": 0,
        "子育て状況不明": np.nan,
        "子の数不詳": np.nan,
        "子供の数不明": np.nan,
        "こども1人": 1,
        "1児": 1,
        "子供1人": 1,
        "子供有り(1人)": 1,
        "子供有り 1人": 1,
        "こども2人": 2,
        "2児": 2,
        "子供2人": 2,
        "子供有り(2人)": 2,
        "こども3人": 3,
        "3児": 3,
        "子供3人": 3,
        "子供有り 2人": 2,
        "子供有り 3人": 3,
        "子供有り(3人)": 3,
        "わからない": np.nan,
        "不明": np.nan,
    }
    
    # データフレームの対象カラムに適用
    df["car"] = df["car"].apply(dict_replace_function, replace_dict=car_replace_dict)
    df["children"] = df["children"].apply(dict_replace_function, replace_dict=children_replace_dict)

    return df


In [75]:
def preprocess_for_last_3_cols(
    df: pd.DataFrame,
) -> pd.DataFrame:
    """
    データフレームに対して前処理を行う。
    
    Args:
        df (pd.DataFrame): 前処理を行うデータフレーム
    
    Returns:
        pd.DataFrame: 前処理後のデータフレーム
    """
    
    # カラムごとの前処理
    df, invalid_values = convert_fullwidth_to_halfwidth_and_extract_invalid(df, "Designation")
    df, unmatched_values = extract_and_convert_to_numeric(df, "MonthlyIncome", "MonthlyIncome_numeric")
    df = customer_info_preprocess(df, "customer_info")
    
    return df

## データの読み込み

In [76]:
# ローカルファイルを読み込む
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
# google colaboratory で実行する場合は以下を有効にする
# from google.colab import drive
# drive.mount('/content/drive')
# train_df = pd.read_csv("/content/drive/mydrive/signate_cup_2024_data/train.csv")
# test_df = pd.read_csv("/content/drive/mydrive/signate_cup_2024_data/test.csv")

In [78]:
train_df = preprocess_for_last_3_cols(train_df)
test_df = preprocess_for_last_3_cols(test_df)
train_df.to_csv("../data/train_preprocessed.csv", index=False)
test_df.to_csv("../data/test_preprocessed.csv", index=False)

不要になったカラムは以下の2つ。デバッグ目的で残しているが、不要な場合は消してください。
- MonthlyIncome
- customer_info